In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
from langchain import hub

prompt_agent = hub.pull('hwchase17/openai-tools-agent')

prompt_agent

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

llm.invoke("HI")

In [ ]:
import subprocess
from langchain_core.tools import tool


@tool
def list_files():
    """List files in current directory"""
    return subprocess.run(["ls"], stdout=subprocess.PIPE).stdout.decode("utf-8")


list_files.invoke({"input": ""})

In [ ]:
llm_with_tool = llm.bind_tools([list_files])

llm_with_tool